## Library

In [211]:
import requests
import random
import time
import math
from copy import deepcopy
from statistics import mean

## Notebook

#### Requisição

In [60]:
def request() -> list:   
    res = requests.get('http://localhost:3000/api/ep1')
    return res.json()

#### Tratamento de Dados

In [62]:
def data_treatment(json:list):
    for i in range(len(json)):
        for key in json[i]:
            if ('prod' in key):
                if (type(json[i][key])!=dict):
                    qnt = json[i][key]
                    if (type(qnt)==str):
                        qnt = float(qnt)                  
                    if (qnt<0):
                        qnt = 0
                        json[i][key] = {'qnt':[qnt], 'price':None} # Mudei para a quantidade ser uma lista
                    else:
                        json[i][key] = {'qnt':[qnt], 'price':None} # Mudei para a quantidade ser uma lista
        try:
            json[i]['date'] = time.ctime(json[i]['date'])
        except:
            continue
    return json

#### Gerar valores aleatórios

In [61]:
def generate_random_price() -> float:
    return round(random.random() * 100, 2)

#### Precificação

##### Função logistica -> A mesma postada no doc pelo professor

In [212]:
def logistic_function(v: float) -> float:
    return 0.5 + (1 / 1 + math.e)**(-1 * v)

##### Função para calcular a variação -> A mesma postada no doc pelo professor

In [194]:
def calc_qnt_variation(last_week: list, current_week: list) -> float:
    return mean(current_week) - mean(last_week) / mean(last_week) if (mean(last_week) != 0) else 0

##### Função que calcula o preço que será utilizado na semana seguinte

In [222]:
def calc_new_price(product_db: dict, sales_week: dict) -> dict:
    for product in product_db:
        if (product_db[product]['qnt'] != None):
            if (product not in sales_week.keys()):
                sales_week[product] = [0]
            product_db[product]['price'] = logistic_function(calc_qnt_variation(product_db[product]['qnt'], sales_week[product])) * product_db[product]['price']
        product_db[product]['qnt'] = sales_week[product]               
    return product_db

##### Função que atualiza o banco de dados (onde consta a quantidade de vendas da última semana e o preço dos produtos) e aplica o preço nos produtos do JSON

In [208]:
def product_pricing(json: list, product_db: dict) -> tuple:
    sales_week = {}

    for transaction in json:
        for key in transaction:
            if ('prod' in key):
                if (key not in product_db.keys()):
                    transaction[key]['price'] = generate_random_price()
                    sales_week[key] = deepcopy(transaction[key]['qnt'])
                    product_db[key] = deepcopy(transaction[key])
                    product_db[key]['qnt'] = None              
                else:
                    transaction[key]['price'] = deepcopy(product_db[key]['price'])
                    if (key not in sales_week.keys()):
                        sales_week[key] = deepcopy(transaction[key]['qnt'])
                    else:
                        sales_week[key] += deepcopy(transaction[key]['qnt'])

    product_db = calc_new_price(product_db, sales_week)

    return json, product_db

#### Main

In [ ]:
product_database = {}
weeks = 5

for week in range(weeks):
    json = data_treatment(request())
    json, product_database = product_pricing(json, product_database)

    print(f'Semana {week}:\n'
          f'Js - {json}\n\n'
          f'Db - {product_database}\n\n\n')